In [ ]:
from dolfin import *
from mshr import *

In [ ]:
# Geometrical parameters
pre_step_length = 10.
after_step_length_final = 40.
pre_step_height = 2.5
after_step_height = 7.5

In [ ]:
# Create mesh
domain = \
    Rectangle(Point(0., 0.), Point(pre_step_length + after_step_length_final, after_step_height)) - \
    Rectangle(Point(0., 0.), Point(pre_step_length, pre_step_height)) - \
    Rectangle(Point(0., 2*pre_step_height), Point(pre_step_length, after_step_height))    

mesh = generate_mesh(domain, 80)
plot(mesh)

In [ ]:
# Create subdomains
subdomains = MeshFunction("size_t", mesh, 2, mesh.domains())
plot(subdomains)

In [ ]:
# Create boundaries
class Inlet(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]) < DOLFIN_EPS

class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (
            (x[0] <= pre_step_length and abs(x[1] - after_step_height + pre_step_height) < DOLFIN_EPS) or
            (x[0] <= pre_step_length and abs(x[1] - pre_step_height) < DOLFIN_EPS) or
            (x[1] <= pre_step_height and abs(x[0] - pre_step_length) < DOLFIN_EPS) or
            (x[1] >= after_step_height - pre_step_height and abs(x[0] - pre_step_length) < DOLFIN_EPS) or
            (x[0] >= pre_step_length and abs(x[1]) < DOLFIN_EPS)
        )
        
class Top(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[1] - after_step_height) < DOLFIN_EPS


boundaries = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundaries.set_all(0)
inlet = Inlet()
inlet_ID = 1
inlet.mark(boundaries, inlet_ID)
bottom = Bottom()
bottom_ID = 2
bottom.mark(boundaries, bottom_ID)
top = Top()
top_ID = 2
top.mark(boundaries, top_ID)

In [ ]:
# Save to xml file
File("channel.xml") << mesh
File("channel_physical_region.xml") << subdomains
File("channel_facet_region.xml") << boundaries
XDMFFile("channel.xdmf").write(mesh)
XDMFFile("channel_physical_region.xdmf").write(subdomains)
XDMFFile("channel_facet_region.xdmf").write(boundaries)